In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Transformando os Dados Processados").getOrCreate()

24/01/05 16:47:16 WARN Utils: Your hostname, airflow resolves to a loopback address: 127.0.1.1; using 192.168.1.79 instead (on interface enp0s3)
24/01/05 16:47:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/05 16:47:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/05 16:47:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
tweets = spark.read.json("/home/eder/Desktop/Airflow/Twitter/data/processed/tweets/*")

tweets.show(n=5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       99|             42|2024-01-03T03:25:...| 35|        10|          7|         27|           94|Tweet fictício cr...|
|       39|             74|2024-01-03T14:38:...| 61|        88|         27|          6|           35|Um terceiro tweet...|
|       51|             23|2024-01-03T02:22:...| 36|        48|         42|         44|           80|Tweet fictício ge...|
|       31|             25|2024-01-03T06:26:...| 79|        58|         49|         44|           61|Outro tweet fictí...|
|      100|             74|2024-01-03T19:32:...|  8|        86|         41|         11|           52|Tweet fictício ge...|
+---------+-----

In [4]:
tweets.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [5]:
from pyspark.sql import functions as f

In [9]:
tweets_conversas = tweets.groupBy(f.to_date("created_at").alias("created_date"))\
                            .agg(
                                f.countDistinct("author_id").alias("n_users"),
                                f.sum("like_count").alias("n_likes"),
                                f.sum("quote_count").alias("n_quotes"),
                                f.sum("reply_count").alias("n_replies"),
                                f.sum("retweet_count").alias("n_retweets"),
                            ).withColumn("weekday", f.date_format("created_date", "E"))

tweets_conversas.show(n=5)

+------------+-------+-------+--------+---------+----------+-------+
|created_date|n_users|n_likes|n_quotes|n_replies|n_retweets|weekday|
+------------+-------+-------+--------+---------+----------+-------+
|  2024-01-02|     26|   1528|    1661|     1431|      1528|    Tue|
|  2023-12-31|     39|   2873|    2403|     2378|      2662|    Sun|
|  2023-12-29|     10|    480|     503|      478|       394|    Fri|
|  2023-12-30|     10|    315|     480|      491|       562|    Sat|
|  2024-01-03|     43|   3182|    3019|     2978|      3109|    Wed|
+------------+-------+-------+--------+---------+----------+-------+
only showing top 5 rows

